# SQL Analysis of CIA Factbook data<br>
by Adam Lang

# Introduction

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:<br>

**-population** - The population as of 2015.<br>
**- population_growth** - The annual population growth rate, as a percentage.<br>
**- area** - The total land and water area.

We'll use the following code to connect our Jupyter Notebook to our database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Overview of the Data

In [2]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics

Here are the descriptions for some of the columns:<br>

**-name** - The name of the country.<br>
**-area-** The country's total area (both land and water).<br>
**-area_land** - The country's land area in square kilometers.<br>
**-area_water** - The country's waterarea in square kilometers.<br>
**-population** - The country's population.<br>
**-population_growth**- The country's population growth as a percentage.<br>
**-birth_rate** - The country's birth rate, or the number of births a year per 1,000 people.<br>
**-death_rate** - The country's death rate, or the number of death a year per 1,000 people.<br>

Let's start by calculating some summary statistics and look for any outlier countries.

In [3]:
%%sql
SELECT MIN(population) AS min_pop,
        MAX(population) AS max_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
        FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


A few things stick out from these summary statistics:<br>

- There's a country with a population of 0<br>
- There's a country with a population of 7256490011 (or more than 7.2 billion people)<br>

Let's use subqueries to zoom in on just these countries without using the specific values.

# Exploring Outliers

In [9]:
%%sql
SELECT * 
    FROM facts
    WHERE population == (SELECT MIN(population)
                    FROM facts
                        );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


This seems to match the [WORLD FACT BOOK Data](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html) which explains how there are no indigenous inhabitants. 

In [10]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population)
                        FROM facts
                        );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.<br>

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

# Summary Statistics Revisited

In [11]:
%%sql
SELECT MIN(population) AS min_pop,
        MAX(population) AS max_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
        FROM facts
        WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Now we can see the max population of a country is 1.4 billion. Which country is this?

In [15]:
%%sql
SELECT name, population AS max_pop
FROM facts
WHERE population == 1367485388;

Done.


name,max_pop
China,1367485388


Now we can see China is the most populated country. 

Which countries have populations above 1 billion?

In [16]:
%%sql
SELECT name, population
FROM facts
WHERE population >= 1000000000;

Done.


name,population
China,1367485388
India,1251695584
World,7256490011


So we can see China and India have the highest population in the world. 

# Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.<br>

We should take care of discarding the row for the whole planet.

In [17]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

# Finding Densely Populated Countries

We will find all countries where the population is above average and the area is below average. 

In [18]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   WHERE name <> 'World'
                   )
AND area < (SELECT AVG(area)
           FROM facts
           WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


The top 5 are:<br>
1. Bangladeseh<br>
2. Germany<br>
3. Iraq<br>
4. Italy<br>
5. Japan

# Next Steps

#### Which Country has the highest growth rate?

In [19]:
%%sql
SELECT name, MAX(population_growth) AS max_pop_growth
FROM facts
WHERE name <> 'World';

Done.


name,max_pop_growth
South Sudan,4.02


It appears South Sudan has the highest growth rate.

#### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [23]:
%%sql
SELECT name, area_water, area_land
FROM facts
WHERE name <> 'World'
ORDER BY area_water DESC
LIMIT 5;


Done.


name,area_water,area_land
Canada,891163,9093507
Russia,720500,16377742
United States,664709,9161966
India,314070,2973193
China,270550,9326410


#### Which countries have a higher than average death rate but a lower than average birth rate?

In [26]:
%%sql
SELECT * 
FROM facts
WHERE death_rate > (SELECT AVG(death_rate)
                   FROM facts
                   WHERE name <> 'World'
                   )
AND birth_rate < (SELECT AVG(birth_rate)
                 FROM facts
                 WHERE name <>'World')
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
15,bb,Barbados,430,430,0,290604,0.31,11.87,8.44,0.3
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
17,be,Belgium,30528,30278,250,11323973,0.76,11.41,9.63,5.87


We can see the top 5 countries with a higher death rate than birth rate above. 

#### Which countries have a higher than average birth rate but a lower than average death rate?

In [27]:
%%sql
SELECT * 
FROM facts
WHERE death_rate < (SELECT AVG(death_rate)
                   FROM facts
                   WHERE name <> 'World'
                   )
AND birth_rate > (SELECT AVG(birth_rate)
                 FROM facts
                 WHERE name <>'World')
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
18,bh,Belize,22966,22806,160,347369,1.87,24.68,5.97,0.0
21,bl,Bolivia,1098581,1083301,15280,10800882,1.56,22.76,6.52,0.62
30,cb,Cambodia,181035,176515,4520,15708756,1.58,23.83,7.68,0.32


We can see the top 5 countries with a higher than average birth rate but lower than average death rate. 

#### What countries have the highest population/area ratio and how does it compare to the previous list we found?

In [35]:
%%sql
SELECT name, population / area AS pop_area_ratio
FROM facts
ORDER BY pop_area_ratio DESC
LIMIT 5;

Done.


name,pop_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191


We can see this is a very different list than what we had for the most densely populated countries which were: Bangladeseh, Germany, Iraq, Italy and Japan.